# refellips demo: Solid/Liquid
This demo presents an example use case for fitting multi-wavelength single-angle ellipsometry data for solid-liquid systems with refellips.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from refnx.analysis import CurveFitter
from refnx.reflect import Slab

In [ ]:
from refellips.dataSE import DataSE, open_EP4file
from refellips.reflect_modelSE import ReflectModelSE
from refellips.objectiveSE import ObjectiveSE
from refellips.structureSE import RI

### Import data

In [ ]:
dname = 'WVASE_example_2nmSiO2_100nmPNIPAM_MultiWavelength_Water.dat'
data = DataSE(data=dname)
np.unique(data.wav)

### Build model

In [ ]:
from refnx.reflect import Linear, Erf

si     = RI('../materials/silicon.csv')
sio2   = RI('../materials/silica.csv')
PNIPAM = RI('../materials/pnipam.csv')
air    = RI('../materials/air.csv')
h2o    = RI('../materials/water.csv')

PNIPAM_layer = PNIPAM(1000)
PNIPAM_layer.thick.setp(vary=True, bounds=(100,1500))
PNIPAM_layer.vfsolv.setp(vary=True, value=0.8, bounds=(0.1,1))
PNIPAM_layer.rough.setp(vary=True, value=100, bounds=(50,300))
PNIPAM_layer.interfaces = Erf()

struc = h2o() | PNIPAM_layer | sio2(20) | si()
struc.contract = 2
struc.solvent= h2o

model = ReflectModelSE(struc, wavelength=658)

# Fitting experimental data

### Model & data pre fit

In [ ]:
fig, ax = plt.subplots()
axt = ax.twinx()

psis = []
deltas = []
for dat in data:
    model.wav = dat._current_wav
    psi, delta = model([65])
    psis.append(psi)
    deltas.append(delta)

data.mask = np.ones_like(data.mask, dtype=bool)
ax.scatter(data.wav, data.psi, color='r')
axt.scatter(data.wav, data.delta, color='b')

ax.plot(data.wav, psis, color='r')
axt.plot(data.wav, deltas, color='b')

ax.set(ylabel='Psi', xlabel='Wavelength (nm)')
axt.set(ylabel='Delta')

In [ ]:
objective = ObjectiveSE(model, data)

In [ ]:
fitter = CurveFitter(objective)
fitter.fit(method='least_squares');

### Model & data post fit

In [ ]:
fig, ax = plt.subplots()
axt = ax.twinx()

psis = []
deltas = []
for dat in data:
    model.wav = dat._current_wav
    psi, delta = model([65])
    psis.append(psi)
    deltas.append(delta)

data.mask = np.ones_like(data.mask, dtype=bool)
p = ax.scatter(data.wav, data.psi, color='r')
d = axt.scatter(data.wav, data.delta, color='b')

ax.plot(data.wav, psis, color='r')
axt.plot(data.wav, deltas, color='b')

ax.legend(handles=[p,d], labels=['Psi', 'Delta'])
ax.set(ylabel='Psi', xlabel='AOI, °')
axt.set(ylabel='Delta')

### Refractive index profile

In [ ]:
fig, ax = plt.subplots()
struc.reverse_structure = True
ax.plot(*struc.sld_profile())
struc.reverse_structure = False
ax.set_ybound(1.3,1.6)
ax.set(ylabel='Refractive index', xlabel='Distance from substrate (Å)')

In [ ]:
for x in fitter.objective.model.parameters:
    print(x)